# QUBO formulation of polynomial equation

In [1]:
from sympy.matrices import Matrix, SparseMatrix

# Use case

To illustrate the metohod we are taking the equation of the two node water system that reads:

$$
 1/2 - x_0 + x_1 = 0 \\
 1 - x_1 = 0 \\
 3 - x_0^2 - x_2 = 0 \\
 -1/2 x_1^2 + x_2 - x_3 = 0 
$$

In [2]:
import numpy as np

def nlfunc(input):
    x0,x1,x2,x3 = input

    def f0():
        return 1/2 - x0 + x1
    
    def f1():
        return 1 - x1
    
    def f2():
        return 3  - x0**2  - x2

    def f3():
        return - 0.5*x1**2 + x2 - x3
    
    return np.array([f0(), f1(), f2(), f3()])

## Classical Solution

The solution of such a small system can be obtained by newton raphson

In [3]:
from quantum_newton_raphson.newton_raphson import newton_raphson

initial_point = np.random.rand(4)
res = newton_raphson(nlfunc, initial_point)
assert np.allclose(nlfunc(res.solution), 0)

/home/nico/QuantumApplicationLab/QuantumNewtonRaphson/quantum_newton_raphson/utils.py:74: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn("spsolve requires A be CSC or CSR matrix format", SparseEfficiencyWarning)


In [4]:
ref_sol = res.solution
ref_sol

array([1.5 , 1.  , 0.75, 0.25])

### Polynomial equation

We first write the polynomial equation as follow (https://www.nature.com/articles/s41598-019-46729-0) 

$$
F(X) = 0
$$

with

$$
F_i = P_i^{(0)} + \sum_j P_{ij}^{(1)}x_j + \sum_{jk} P_{ijk}^{(2)}x_j x_k  + \sum_{jkl} P_{ijkl}^{(3)}x_j x_k x_l = 0
$$

To solve the system we minimize the residual sum of square

$$
\chi^2 = [P^{(0)} + P^{(1)} X + P^{(2)} X X.T + P^{(3)} X X X ]^2
$$


In [53]:
import numpy as np
import sparse
def define_matrices():
    
    # system of equations
    num_equations = 4
    num_variables = 4

    weight_quadratic_eq = 0.1

    P0 = np.zeros(num_equations)
    P0[0] = 1/2
    P0[1] = 1
    P0[2] = 3*weight_quadratic_eq
    P0[3] = 0*weight_quadratic_eq

    P1 = np.zeros((num_equations, num_variables))
    P1[0, 0] = -1
    P1[0, 1] =  1

    P1[1, 1] = -1

    P1[2, 2] = -1*weight_quadratic_eq

    P1[3, 2] =  1 *weight_quadratic_eq
    P1[3, 3] = -1*weight_quadratic_eq
   

    P2 = np.zeros((num_equations, num_variables, num_variables))
    P2[2, 0, 0] = -1*weight_quadratic_eq
    P2[3, 1, 1] = -1/2*weight_quadratic_eq

    # P3 = np.zeros((num_equations, num_variables, num_variables, num_variables))
    # P3[2,2,2,2] = -1


    return sparse.COO(P0), sparse.COO(P1), sparse.COO(P2) #, sparse.COO(P3)

matrices = define_matrices()

## 3. Solving the system

We will use here the `SimulatedAnnealingSampler` to be able to run that code locally. Quantum solvers are available through the Leap cloud service.

In [54]:
from qubols.qubo_poly import QUBO_POLY
from qubols.qubo_poly_mixed_variables import QUBO_POLY_MIXED
from qubols.encodings import PositiveQbitEncoding, RangedEfficientEncoding
from qubols.solution_vector import SolutionVector_V2 as SolutionVector
from qubols.mixed_solution_vector import MixedSolutionVector_V2 as MixedSolutionVector

from dwave.samplers import SimulatedAnnealingSampler
from dwave.samplers import SteepestDescentSolver
from dwave.samplers import TabuSampler
from dimod import ExactSolver

nqbit = 5
step = 2/(2**nqbit-1)
encoding1 = PositiveQbitEncoding(nqbit = nqbit, step=step, offset=0, var_base_name='x')
# encoding1 = RangedEfficientEncoding(nqbit=nqbit, range=5, offset=5, var_base_name='x')
sol_vec1 = SolutionVector(2, encoding=encoding1)

nqbit = 5
step = 2/(2**nqbit-1)
encoding2 = PositiveQbitEncoding(nqbit = nqbit, step=step, offset=0, var_base_name='x')
# encoding2 = RangedEfficientEncoding(nqbit=nqbit, range=5, offset=5, var_base_name='x')
sol_vec2 = SolutionVector(2, encoding=encoding2)

sol_vec = MixedSolutionVector([sol_vec1,sol_vec2])

sampler = SimulatedAnnealingSampler()

options = {'num_reads':1000, 'sampler':sampler}
qubo = QUBO_POLY_MIXED(sol_vec, options)

In [55]:
np.sort(encoding1.get_possible_values())

array([0.   , 0.065, 0.129, 0.194, 0.258, 0.323, 0.387, 0.452, 0.516,
       0.581, 0.645, 0.71 , 0.774, 0.839, 0.903, 0.968, 1.032, 1.097,
       1.161, 1.226, 1.29 , 1.355, 1.419, 1.484, 1.548, 1.613, 1.677,
       1.742, 1.806, 1.871, 1.935, 2.   ])

In [56]:
matrices = tuple(sparse.COO(m) for m in matrices)

bqm = qubo.create_bqm(matrices, strength=10000)

# sample
sampleset = qubo.sample_bqm(bqm, num_reads=10000)

# decode
qubo.verify_quadratic_constraints(sampleset.lowest())
sol  = qubo.decode_solution(sampleset.lowest().record[0][0])
sol = np.array(sol).reshape(-1)

data_ref, eref = qubo.compute_energy(ref_sol, bqm)
data_sol, esol = qubo.compute_energy(sol, bqm)


np.set_printoptions(precision=3)

print('prec: ', encoding1.get_average_precision(), encoding2.get_average_precision())
print('\n')

print('ref : ', np.array(ref_sol)) 
print('sol : ', sol)
print('diff: ', ref_sol - sol)
print('\n')

print('encoded_ref: ', np.array(data_ref[0]))
print('encoded_sol: ', np.array(data_sol[0]))
print('diff       : ', np.array(data_ref[0]) - np.array(data_sol[0]))
print('\n')
print('eref: ', eref)
print('esol: ', esol)
print('\n')
print('res_ref: ', np.linalg.norm(nlfunc(data_ref[0])))
print('res_sol: ', np.linalg.norm(nlfunc(data_sol[0])))



prec:  0.06451612903225806 0.06451612903225806


ref :  [1.5  1.   0.75 0.25]
sol :  [1.484 0.968 0.839 0.387]
diff:  [ 0.016  0.032 -0.089 -0.137]


encoded_ref:  [1.484 1.032 0.774 0.258]
encoded_sol:  [1.484 0.968 0.839 0.387]
diff       :  [ 0.     0.065 -0.065 -0.129]


eref:  -1.3366096060604533
esol:  -1.3386800300201653


res_ref:  0.06505306758847451
res_sol:  0.05678808027274856
